# RNN Seq2Seq to Attention and Forward!

Last we saw how RNN in Seq2Seq models faces the following bottlenecks:
1. Single vector representing the entire information captured in the encoder step which is unable to represent the word level information in the sequence properly.
2. Relevant word distance from the word being generated. 
3. Sequential processing due to dependence on previous time step calculations.

We also saw how the first and second issue were resolved with the introduction of attention to the RNN-based Seq2Seq model. The inclusion of attention weighted vectors (softmaxed over the dot product of decoder hidden state and all the encoder hidden states) acting as a soft focus/weighing mechanism to consider the importance and context of each word in the sequence that came as a precursor to the word being generated now, acted as a huge improvement to the purely RNN based Seq2Seq models.

But we are yet to resolve the sequential processing problem which turns out to be a major bottleneck in model training from a time complexity, implementation and engineering perspective.